In [4]:
from pycox.evaluation import EvalSurv
import torch
import torchtuples as tt
import os
import pickle
from lifelines.utils import concordance_index

from utils import dataset_name
from eval_utils import MODEL_CLASS, get_results, DL_MODELS, ML_MODELS, MODELS
import numpy as np
import pandas as pd
np.random.seed(42)
_ = torch.manual_seed(42)


print(f"Experiments over {dataset_name}")

Experiments over GBSG


In [5]:
train = pd.read_csv(f"../../datasets/train/{dataset_name}.csv")
test = pd.read_csv(f"../../datasets/test/{dataset_name}.csv")

X, y = train.drop(["event", "duration"], axis=1), train[["event", "duration"]]
X_test, y_test = test.drop(["event", "duration"], axis=1), test[["event", "duration"]]

# Finding the best parameters for each model and run the models over the test data

In [6]:
results = pd.DataFrame()
     
for model in MODELS:
    stats = pickle.load( open( f"statistics/{model}/best_model.pkl", "rb" ))
    df = pd.DataFrame(stats, index=[model])
    results = pd.concat([results, df], axis=0)
    
c_index_df = results[["c_index", "c_index_std", "c_index_params"]].sort_values("c_index", ascending=False)
concordance_td_df = results[["concordance_td", "concordance_td_std", "concordance_td_params"]].sort_values("concordance_td", ascending=False)
ibs_df = results[["ibs", "ibs_std", "ibs_params"]].sort_values("ibs", ascending=True)

## `c index`

In [7]:
c_index_df

,c_index,c_index_std,c_index_params
deep_surv,0.684238,0.016806,"deep_surv_dropout_0.1_num_nodes_[32, 32]_activ..."
cox_time,0.679852,0.016667,"cox_time_dropout_0.1_num_nodes_[16, 32, 64, 64..."
rsf,0.679400,0.012270,rsf_n_estimators_100_max_depth_10_min_samples_...
gbst,0.678070,0.015949,gbst_learning_rate_0.01_n_estimators_50_subsam...
reg_coxph,0.660820,0.017125,reg_coxph_l1_ratio_0.1_tol_1e-09_max_iter_1000...
pc_hazard,0.578025,0.048076,"pc_hazard_dropout_0.1_num_nodes_[16, 16, 16, 6..."
deep_hit,0.493453,0.039836,"deep_hit_dropout_0.3_num_nodes_[3, 5]_activati..."


In [15]:
results = []
column = "c_index_params"


for model_name, params in zip(c_index_df.index, c_index_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

cox_time 0.6629
rsf 0.6599
gbst 0.6578
reg_coxph 0.6562
deep_surv 0.6561
deep_hit 0.5274
pc_hazard 0.4168


___

## `concordance td`

In [10]:
concordance_td_df

,concordance_td,concordance_td_std,concordance_td_params
deep_surv,0.683970,0.016733,"deep_surv_dropout_0.1_num_nodes_[32, 32]_activ..."
gbst,0.677993,0.015886,gbst_learning_rate_0.01_n_estimators_50_subsam...
cox_time,0.677787,0.016194,"cox_time_dropout_0.1_num_nodes_[16, 32, 64, 64..."
rsf,0.674173,0.015883,rsf_n_estimators_100_max_depth_10_min_samples_...
deep_hit,0.662318,0.006979,"deep_hit_dropout_0.1_num_nodes_[28, 28, 100, 2..."
reg_coxph,0.660515,0.016824,reg_coxph_l1_ratio_0.1_tol_1e-09_max_iter_1000...
pc_hazard,0.573562,0.042601,pc_hazard_dropout_0.1_num_nodes_[100]_activati...


In [16]:
results = []
column = "concordance_td_params"


for model_name, params in zip(concordance_td_df.index, concordance_td_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

deep_hit 0.6669
rsf 0.6595
cox_time 0.6579
gbst 0.6578
reg_coxph 0.6562
deep_surv 0.6561
pc_hazard 0.5551


## `integrated brier score`

In [12]:
ibs_df

,ibs,ibs_std,ibs_params
deep_hit,0.166143,0.016634,"deep_hit_dropout_0.3_num_nodes_[16, 32, 64, 64..."
deep_surv,0.174653,0.007440,deep_surv_dropout_0.1_num_nodes_[100]_activati...
cox_time,0.176549,0.007609,"cox_time_dropout_0.1_num_nodes_[32, 128, 128]_..."
rsf,0.176950,0.006720,rsf_n_estimators_100_max_depth_10_min_samples_...
gbst,0.179145,0.006892,gbst_learning_rate_0.1_n_estimators_50_subsamp...
reg_coxph,0.184000,0.008836,reg_coxph_l1_ratio_0.1_tol_0.1_max_iter_1000_v...
pc_hazard,0.644894,0.015739,"pc_hazard_dropout_0.1_num_nodes_[16, 16, 16, 6..."


In [17]:
results = []
column = "ibs_params"


for model_name, params in zip(ibs_df.index, ibs_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))

for m,val in sorted(results, key=lambda x: x[1], reverse=False):
    print(m, np.round(val,4))

cox_time 0.1768
deep_surv 0.1783
rsf 0.1785
gbst 0.1788
reg_coxph 0.1798
deep_hit 0.3711
pc_hazard 0.3752
